# Set Up

## imports

In [10]:
import pandas as pd
import numpy as np

import os
from tqdm import tqdm
import warnings

import plotly.express as px
import plotly.io as po
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

import sympy #미분
import math # arctangent; math.atan 사용 목적
import statistics

## Directory

In [11]:
folder_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/17_드론궤적분석자료_2점분합류_금호JC_부산방향/B_affected'

In [12]:
# 중간결과 저장용
angle_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/39_22-13차 회의 준비/02_연속분류구간_위험운전행동_재산출/to_Busan/B/save'

In [13]:
# 최종결과 파일 저장 경로
save_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/39_22-13차 회의 준비/02_연속분류구간_위험운전행동_재산출/to_Busan/B/decision'

## Load Datasets

In [14]:
folder_name = '01_density_by_target_LOS_concated_lanechange' # 파일 들어있는 폴더

file_dir = os.path.join(folder_dir, folder_name)

In [15]:
file_list = os.listdir(file_dir)
file_list = file_list[3:]

file_list

['02_4_A.csv', '02_4_B.csv', '02_4_C.csv']

In [16]:
df_list = []
num_LOS_list = []

for file in file_list:
    num_LOS = file[0:6]
    
    file_path = os.path.join(file_dir, file)
    
    globals()[f'target{num_LOS}'] = pd.read_csv(file_path, encoding = 'cp949')
    
    df_list.append(globals()[f'target{num_LOS}'])
    num_LOS_list.append(num_LOS)

In [17]:
df_list[0].head(3) # 01_1_A.csv

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Vehicle ID,Frame ID,Total Frames,Global Time (Epoch Time),Local X (m),...,Spacing Headway,Time Headway,Bad Object,Special Car,Lane Class,Vehicle Movement,Lane_record,Lane_00,Lane_99,Lane_change
0,0,0,806,812,812,48,807,222,46826900,161.817703,...,0.0,0.0,0,0,상행본선,2.145787,U5,U5,U5,NaN
1,1,1,807,813,813,51,807,212,46826900,171.584625,...,0.0,0.0,0,0,상행본선,2.134770,U3_U2,U3,U2,NaN
2,2,2,808,814,814,55,807,172,46826900,143.477768,...,0.0,0.0,0,0,상행본선,2.313869,U2,U2,U2,NaN


# 급진로변경 표시
* **급진로변경**
    * 속도가 30km/h 이상에서
    * 진행방향이 좌측 또는 우측 6도/sec 이상으로 차로변경하고
    * 5초간 누적각도가 +-2도/sec 이하
    * 가감속이 초당 +=2km/h이하인 경우
* **tan(x)**
    * tan6 : `math.tan(math.pi/60)`
    * tan(-6) : - 0.5773502691896257
    * tan2 : `math.tan(math.pi/90)`
    * tan(-2) : -tan2
* **과정**
    1. 각 차량의 각도차를 구한다.
    2. 누적각도를 구한다.
    3. 급진로변경 판정
    4. 원본 .csv 파일 저장
    5. 해당 지점을 산점도로 표시한 .plotly 파일 저장
    6. 급진로변경 건수를 저장한 피벗테이블 저장

## Load Dataset

### 차량별 좌표별 이동 벡터 구하기
* A : 각 대상지별, 서비스수준별 차량 데이터에 대하여, 현재 (x,y) 좌표와 다음(x', y') 좌표에 대하여 기울기를 구한다.

In [18]:
warnings.filterwarnings(action = 'ignore') # pandas 경고메시지 숨기기

slope_df_list = []

for df, num_LOS in tqdm(zip(df_list, num_LOS_list)):
    # 차량별 이동 동선을 써야 함
    vehicle_list = df['Vehicle ID'].unique()
    
    slope_df = pd.DataFrame()
    
    for veh in vehicle_list: # 각 차량별로
        veh_df = df[df['Vehicle ID'] == veh]
        veh_df['X_next'] = veh_df['Local X (m)'].shift(-1) # 각 차량의 다음 점의 x좌표. 없으면 None이겠지
        veh_df['Y_next'] = veh_df['Local Y(m)'].shift(-1) # 각 차량의 다음 점의 Y좌표. 없으면 None이겠지?
        
        veh_df['slope_veh'] = (veh_df['Y_next'] - veh_df['Local Y(m)'])/(veh_df['X_next'] - veh_df['Local X (m)']) # 각 차량의 벡터. 전 이동지점과 다음이동지점에 대한 것임
        
        slope_df = pd.concat([slope_df, veh_df])
    
    slope_df['slope_veh_before'] = slope_df['slope_veh'].shift(1) # 1칸 이전 벡터를 끌고옴. lag 데이터임
    
    slope_df_list.append(slope_df)
    
    save_slope_file = f'save1_slope_{num_LOS}.csv'
    save_slope_path = os.path.join(angle_dir, save_slope_file)
    
    slope_df.to_csv(save_slope_path, encoding = 'cp949')

3it [00:17,  5.88s/it]


In [19]:
slope_df.head(3)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Vehicle ID,Frame ID,Total Frames,Global Time (Epoch Time),Local X (m),...,Lane Class,Vehicle Movement,Lane_record,Lane_00,Lane_99,Lane_change,X_next,Y_next,slope_veh,slope_veh_before
0,0,0,15753,15986,1277,10059,101146,221,46838200,165.487534,...,상행본선,2.116368,U5,U5,U5,NaN,167.605194,30.555534,0.043501,NaN
12,7,7,15760,15993,1284,10059,101149,224,46838300,167.605194,...,상행본선,2.119663,U5,U5,U5,NaN,169.721207,30.639816,0.039831,0.043501
20,14,14,15767,16000,1291,10059,101152,227,46838400,169.721207,...,상행본선,2.117690,U5,U5,U5,NaN,171.834961,30.711826,0.034067,0.039831


## 판정을 위한 변수 생성
* 직전과 다음 벡터 사이의 사잇각을 탄젠트 등을 이용하여 구하고, .csv 파일로 저장

* **급진로변경**
    * 속도가 30km/h 이상에서, 
    * 진행방향이 좌측 또는 우측 6도/sec 이상이고(*차로변경 조건 폐기*) 
    * 5초간 누적각도가 +-2도/sec 이하, 
    * 가감속이 초당 +=2km/h(**0.555556m/s**)이하인 경우
* `tan6` : `math.tan(math.pi/30)`
* `tan(-6)` = `-tan6`
* **tan6 < S | S < tan(-6) : `overangle`**
* 각 차량의 각도차(`tangap`)를 구한다.
* 지난 5초간의 누적각도(`cum_angle`)를 구할 시
    * `math.atan(x)` 사용하여 구하기

### 필요 함수 정의

In [20]:
def make_tangap(slope_veh1, slope_veh2):
    """차량의 움직임 벡터(slope_veh) 전후값을 사용하여,
    매 점에서의 두 벡터간 차이에 의해 발생하는 사잇각의 tangent 값을 구하는 함수
    --- tan(A-B) = (tanA - tanB)/(1 + tanA * tanB) 임을 유의 ---
    
    slope_veh1 :: 해당 차량의 직전 이동 벡터
    slope_veh2 :: 해당 차량의 이동 벡터"""
    
    tangap = (slope_veh2 - slope_veh1)/(1 + slope_veh1 * slope_veh2)
    
    return tangap

In [21]:
def make_tangap_rad(tangap):
    """사잇각의 탄젠트값(tangap)에 arctangent를 적용,
    해당 사잇각의 radian값 = 실제 각도를 얻어내는 함수"""
    
    rad_tangap = math.atan(tangap)
    
    return rad_tangap

In [22]:
def overangle(rad_tangap):
    """
    벡터차이의 라디안 값, 즉 벡터간 각도가 좌, 우 6도 이상으로 각도변경했는지(overangle) 판정하는 함수
    ---tan6도 = tan(math.pi/30) 임에 유의 ---
    ---abs()는 절대값을 반환하는 함수임 ---
    
    """
    tan_plus_6 = math.tan(math.pi/30) # tan(6도)
    tan_minus_6 = (-1) * tan_plus_6 # tan(-6도)
    
    if rad_tangap >= tan_plus_6 or rad_tangap <= tan_minus_6: # 좌로 6도, 혹은 우로 6도 이상 틀어졌을 시
        return 'overangle' # 오버앵글 판정
    
    else:
        pass

In [23]:
def vehicle_direction(rad_tangap):
    """
    차량이 도로선형에 비해 상대적으로 왼쪽으로 갔는지/오른쪽으로 갔는지 판정해주는 함수
    --- tangap > 0 : 차로를 중심으로 오른쪽
    --- tangap < 0 : 차로를 중심으로 왼쪽"""
    
    if rad_tangap > 0:
        return 'right'
    
    elif rad_tangap < 0:
        return 'left'
    
    elif rad_tangap == 0:
        return 'straight'
    
    else:
        pass

### tan 계산
* 여러 컬럼을 함수에 적용하기 : `데이터프레임이름[['변수1', '변수2']].apply(lambda x: 함수이름(*x), axis = 1)`
* 0.1초 단위의 tangent 값, tangent 값을 바탕으로 한 각도(rad) 값 구하기

In [24]:
for slope_df, num_LOS in tqdm(zip(slope_df_list, num_LOS_list)): # 차량벡터들을 땄다.
    
    slope_df['tangap'] = slope_df[['slope_veh', 'slope_veh_before']].apply(lambda x : make_tangap(*x), axis = 1) # 차량벡터 전후값 사이 탄젠트값 생성
    slope_df['rad_tangap'] = slope_df['tangap'].apply(make_tangap_rad) # tangap 의 라디안 값 생성. 이 라디안 값들끼리 더할 수 있다.
    slope_df['vehicle_direction'] = slope_df['rad_tangap'].apply(vehicle_direction) # 도로선형을 중심으로 상대적으로 차량이 어느쪽으로 회전중인지 판정

3it [00:00,  7.39it/s]


In [25]:
slope_df.head(3)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Vehicle ID,Frame ID,Total Frames,Global Time (Epoch Time),Local X (m),...,Lane_00,Lane_99,Lane_change,X_next,Y_next,slope_veh,slope_veh_before,tangap,rad_tangap,vehicle_direction
0,0,0,15753,15986,1277,10059,101146,221,46838200,165.487534,...,U5,U5,NaN,167.605194,30.555534,0.043501,NaN,NaN,NaN,None
12,7,7,15760,15993,1284,10059,101149,224,46838300,167.605194,...,U5,U5,NaN,169.721207,30.639816,0.039831,0.043501,0.003664,0.003664,right
20,14,14,15767,16000,1291,10059,101152,227,46838400,169.721207,...,U5,U5,NaN,171.834961,30.711826,0.034067,0.039831,0.005755,0.005755,right


### 시간 변수, 누적 각도 생성
* 각 차량의 지난 5초(5 * 프레임레이트만큼의 프레임ID) 간의 누적각도를 구함
* 관련 변수들의 계산 : 
    * `rad_tangap_abs` : rad_tangap의 절대값
    * `cumsum_rad_abs` : 차량별 "총" 누적 각도. rad_tangap 절대값의 합
    * `cumsum_rad_abs_final_1s` : 차량별 1초 누적각도. rad_tangap의 최근 1초간의 합
    * `cumsum_rad_abs_final_1s_tan` :  `cumsum_rad_1s`를 tan으로 환산한 값
    * `cumsum_rad_abs_final_5s` : 차량별 5초 누적각도. rad_tangap의 최근 5초간의 합
    * `cumsum_rad_abs_final_5s_tan` :  `cumsum_rad_5s`를 tan으로 환산한 값

In [26]:
frame_rate = 30 # 초당 프레임 계산 : 3프레임 당 0.1초, 1초당 30프레임임

In [27]:
cumsum_df_list = []

for slope_df, num_LOS in tqdm(zip(slope_df_list, num_LOS_list)):
    # 주의 :: 누적각도인 cumsum_rad는 차량별(Vehicle ID)로 구해야 한다.
    
    slope_df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1'], axis = 1, inplace = True) # 불필요한 컬럼 제거

    cumsum_df = pd.DataFrame()
    
    veh_list = slope_df['Vehicle ID'].unique() # 각 테이블의 차량리스트 생성
    
    for veh in veh_list:
        veh_df = slope_df[slope_df['Vehicle ID'] == veh] # 각 차량별 궤적 데이터프레임 생성
        
        frame_max = veh_df['Frame ID'].max() # 해당 차량이 찍힌 마지막 프레임
        frame_min = veh_df['Frame ID'].min() # 해당 차량이 찍힌 최초 프레임
        
        veh_runtime = (frame_max - frame_min)/frame_rate # 각 차량별 총 주행시간(초)을 구함. 
        
        # cumsum_rad_ : 누적각도합(총계) 구하기
        veh_df['cumsum_rad'] = veh_df['rad_tangap'].cumsum(skipna = True) # 라디안 값의 누적값을 구하기
        
        # 데이터는 15프레임 = 0.5초 단위로 찍혀있음. 1초 = 2행, 5초 = 10행에 해당함
        veh_df['cumsum_rad_before_1s'] = veh_df['cumsum_rad'].shift(2) ########## 1초 = 30프레임만큼 back
        veh_df['cumsum_rad_before_5s'] = veh_df['cumsum_rad'].shift(10) ########## 5초 = 150프레임만큼 back

        veh_df['cumsum_rad_final_1s'] = veh_df['cumsum_rad'] - veh_df['cumsum_rad_before_1s'] # 1초 누적 값의 라디안 값임
        veh_df['cumsum_rad_final_5s'] = veh_df['cumsum_rad'] - veh_df['cumsum_rad_before_5s'] # 5초 누적 값의 라디안 값임
        
        cumsum_df = pd.concat([cumsum_df, veh_df])
    
    cumsum_df_list.append(cumsum_df)
    
    cumsum_df_file = f'save2_cumsum_{num_LOS}.csv'
    cumsum_df_path = os.path.join(angle_dir, cumsum_df_file)
    
    cumsum_df.to_csv(cumsum_df_path, encoding = 'cp949')

3it [00:17,  5.91s/it]


### 판정 함수 설정
* `decision` : 급진로변경(`rapid_change`) 또는 급앞지르기(`rapid_passing`) 판정판정
* **급진로변경**
    * 속도가 30km/h 이상에서, 
    * 진행방향이 좌측 또는 우측 6도/sec 이상으로 차로변경하고,
    * 5초간 누적각도가 +-2도/sec 이하, 
    * 가감속이 초당 +-2km/h(**0.555556m/s**)이하인 경우
* **급앞지르기**
    * 속도가 30km/h 이상에서, 
    * 진행방향이 좌측 또는 우측 6도/sec 이상으로 차로변경하고,
    * 5초간 누적각도가 +-2도/sec 이하, 
    * 가속이 초당 +3km/h(**0.8333333m/s**)이하인 경우
* 단위 관련 : 360도 = 2*pi, 1도 = pi/180 rad.
    * 6도 : pi/30 rad
    * 2도 : pi/90 rad

In [28]:
def decision(Lane_change, velocity, acceleration, cumsum_rad_1s, cumsum_rad_5s):
    if Lane_change == 'Change':
        if velocity >= 30:
            if (cumsum_rad_1s >= np.pi/30) or (cumsum_rad_1s <= -np.pi/30):
                if (cumsum_rad_5s >= -np.pi/90) and (cumsum_rad_5s <= np.pi/90):
                    if (acceleration <= 2*1000/3600) and (acceleration >= -2*1000/3600): # 가감속 : 초속으로 변환
                        return 'rapid_change'
                    elif (acceleration >= 3*1000/3600):
                        return 'rapid_passing'
                    else:
                        return None
        
                else:
                    return None
            else:
                return None
        else:
            return None
    else:
        return None

In [29]:
def decision_1(velocity):
    if velocity >= 30:
        return '속도 30 이상'
    else:
        return None

In [30]:
def decision_2_1(acceleration):
    if (acceleration <= 2*1000/3600) and (acceleration >= -2*1000/3600): # 가감속 : 초속으로 변환. -2~2km/h 사이여야 한다. 이를 초속으로 변환할 필요
        return '가속도 2 이하'
    else:
        return None

In [31]:
def decision_2_2(acceleration):
    if (acceleration >= 3*1000/3600): # 가감속 : 초속으로 변환. -2~2km/h 사이여야 한다. 이를 초속으로 변환할 필요
        return '가속도 3 이상'
    else:
        return None

In [32]:
def decision_3(cumsum_rad_1s):
    if (cumsum_rad_1s >= np.pi/30) or (cumsum_rad_1s <= -np.pi/30):
        return '1초간 6도 이상 변화'
    else:
        pass

In [33]:
def decision_4(cumsum_rad_5s):
    if (cumsum_rad_5s >= -np.pi/90) and (cumsum_rad_5s <= np.pi/90):
        return '6초간 2도 이하 변화'
    else:
        pass

### 판정: 급앞지르기/급진로변경
`cumsum_df['decision'] = cumsum_df[['Vehicle Velocity(km/h)', 'Vehicle Acceleration', 'cumsum_rad_final_1s', 'cumsum_rad_final_5s']].apply(lambda x : decision(*x), axis = 1)`

In [34]:
for cumsum_df, num_LOS in tqdm(zip(cumsum_df_list, num_LOS_list)):
    
    cumsum_df['decision_1'] = cumsum_df['Vehicle Velocity(km/h)'].apply(decision_1)
    cumsum_df['decision_2_1'] = cumsum_df['Vehicle Acceleration'].apply(decision_2_1)
    cumsum_df['decision_2_2'] = cumsum_df['Vehicle Acceleration'].apply(decision_2_2)
    cumsum_df['decision_3'] = cumsum_df['cumsum_rad_final_1s'].apply(decision_3)
    cumsum_df['decision_4'] = cumsum_df['cumsum_rad_final_5s'].apply(decision_4)
    
    cumsum_df['decision'] = cumsum_df[['Lane_change', 'Vehicle Velocity(km/h)', 'Vehicle Acceleration', 'cumsum_rad_final_1s', 'cumsum_rad_final_5s']].apply(lambda x : decision(*x), axis = 1)
    
    cumsum_df_file = f'decision_rowfiles_{num_LOS}.csv'
    cumsum_df_path = os.path.join(angle_dir, cumsum_df_file)
    
    cumsum_df.to_csv(cumsum_df_path, encoding = 'cp949')

3it [00:01,  1.55it/s]


# Pivoted Data

In [35]:
harmonic = lambda x: statistics.harmonic_mean(list(x))
harmonic .__name__ = 'harmonic'

unq = lambda x: len(x.unique())
unq .__name__ = 'unq'

In [36]:
cumsum_df['decision'].unique()

array([None], dtype=object)

In [37]:
for cumsum_df, num_LOS in tqdm(zip(cumsum_df_list, num_LOS_list)):
    
    pvdf = pd.pivot_table(cumsum_df,
                          index = ['Lane_record', 'decision'],
                          values = ['Vehicle ID', 'Vehicle Velocity(km/h)', 'Vehicle Acceleration'],
                          aggfunc = {'Vehicle ID': unq,
                                     'Vehicle Velocity(km/h)': harmonic,
                                     'Vehicle Acceleration': 'mean',
                                     'decision' : 'count'})
    
    if len(pvdf) > 0:
        save_pv_name = f'Decision_Pivot_{num_LOS}.csv'
        save_pv_path = os.path.join(save_dir, save_pv_name)
        
        pvdf.to_csv(save_pv_path, encoding = 'cp949')
    
    else:
        pass

3it [00:00, 55.67it/s]
